In [1]:
import numpy as np
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from scipy.optimize import minimize
import warnings
warnings.filterwarnings('ignore')

# Historical data for Function 8 (weeks 1-3)
X = np.array([
    [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1],           # Week 1 input
    [0.12, 0.09, 0.11, 0.1, 0.08, 0.13, 0.11, 0.09],    # Week 2 input
    [0.288486, 0.250604, 0.018629, 0.285988, 0.135625, 0.216996, 0.248538, 0.295101]   # Week 3 input
])

y = np.array([
    9.5423,           # Week 1 output
    9.55359,          # Week 2 output
    9.5483251064884   # Week 3 output
])

print("Function 8 Training Data:")
print("="*60)
for i in range(len(X)):
    print(f"Week {i+1}: Input = {X[i]}")
    print(f"         Output = {y[i]:.8f}")
print()

# Method 1: Linear Regression
print("Method 1: Linear Regression")
print("-"*60)
lr_model = LinearRegression()
lr_model.fit(X, y)

# Method 2: Gradient Boosting
print("Method 2: Gradient Boosting")
print("-"*60)
gb_model = GradientBoostingRegressor(n_estimators=100, random_state=42)
gb_model.fit(X, y)

# Method 3: Random Forest
print("Method 3: Random Forest")
print("-"*60)
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X, y)

# Method 4: Neural Network
print("Method 4: Neural Network")
print("-"*60)
nn_model = MLPRegressor(hidden_layer_sizes=(4, 4), max_iter=2000, random_state=42)
nn_model.fit(X, y)

# Method 5: Gaussian Process
print("Method 5: Gaussian Process")
print("-"*60)
kernel = C(1.0, (1e-3, 1e3)) * RBF(1.0, (1e-3, 1e3))
gp_model = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=10, random_state=42)
gp_model.fit(X, y)

# Method 6: Bayesian Optimization using GP
print("Method 6: Bayesian Optimization")
print("-"*60)

def acquisition_function(x, model):
    """Expected Improvement acquisition function"""
    x = x.reshape(1, -1)
    mu, sigma = model.predict(x, return_std=True)
    # We want to maximize, so we negate for minimization
    return -mu[0]

# Use Bayesian Optimization to find best input
best_score = float('-inf')
best_input_bo = None

# Try multiple random starts for optimization
for _ in range(40):
    x0 = np.random.uniform(0.001, 0.5, 8)
    
    result = minimize(
        lambda x: acquisition_function(x, gp_model),
        x0,
        method='L-BFGS-B',
        bounds=[(0.001, 1.0)] * 8
    )
    
    if result.success:
        pred_value = gp_model.predict(result.x.reshape(1, -1))[0]
        if pred_value > best_score:
            best_score = pred_value
            best_input_bo = result.x

print(f"Bayesian Optimization found: {best_input_bo}")
print()

# Test candidate inputs
candidates = np.array([
    [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1],
    [0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15],
    [0.12, 0.12, 0.12, 0.12, 0.12, 0.12, 0.12, 0.12],
    [0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08],
    [0.11, 0.09, 0.1, 0.12, 0.08, 0.13, 0.1, 0.11],  # Varied around 0.1
    [0.13, 0.11, 0.09, 0.1, 0.12, 0.08, 0.14, 0.1],  # Different variation
    best_input_bo  # Add BO result
])

print("\nPredictions for candidate inputs:")
print("="*80)
print(f"{'Input (first 4 values)':<30} {'LR':<12} {'GB':<12} {'RF':<12} {'NN':<12} {'GP':<12}")
print("-"*80)

all_predictions = []
for candidate in candidates:
    lr_pred = lr_model.predict([candidate])[0]
    gb_pred = gb_model.predict([candidate])[0]
    rf_pred = rf_model.predict([candidate])[0]
    nn_pred = nn_model.predict([candidate])[0]
    gp_pred = gp_model.predict([candidate])[0]
    
    avg_pred = (lr_pred + gb_pred + rf_pred + nn_pred + gp_pred) / 5
    all_predictions.append(avg_pred)
    
    cand_str = f"[{candidate[0]:.3f}, {candidate[1]:.3f}, {candidate[2]:.3f}, {candidate[3]:.3f}...]"
    print(f"{cand_str:<30} {lr_pred:<12.6f} {gb_pred:<12.6f} {rf_pred:<12.6f} {nn_pred:<12.6f} {gp_pred:<12.6f}")

print()
print("Ensemble Predictions (average of all models):")
print("-"*80)
for i, candidate in enumerate(candidates):
    cand_str = f"[{candidate[0]:.3f}, {candidate[1]:.3f}, {candidate[2]:.3f}, {candidate[3]:.3f}...]"
    print(f"{cand_str:<30} → {all_predictions[i]:.8f}")

# Choose best based on ensemble
best_idx = np.argmax(all_predictions)
best_input = candidates[best_idx]

print()
print("="*80)
print(f"RECOMMENDED Week 4 submission for Function 8:")
print(f"{best_input}")
print(f"Expected output: {all_predictions[best_idx]:.8f}")
print("="*80)

# Additional analysis
print("\nAdditional Analysis:")
print("-"*80)
print("Observation: Function 8 outputs are all very close (around 9.54-9.55)")
print("Week 1: [0.1] (uniform) → 9.5423")
print("Week 2: [varied ~0.1] → 9.55359 (BEST so far)")
print("Week 3: [more varied] → 9.5483")
print("Pattern: Small variations around 0.1-0.15 range, with Week 2's approach best")
print("The function seems relatively stable - maximizing requires fine-tuning")

Function 8 Training Data:
Week 1: Input = [0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]
         Output = 9.54230000
Week 2: Input = [0.12 0.09 0.11 0.1  0.08 0.13 0.11 0.09]
         Output = 9.55359000
Week 3: Input = [0.288486 0.250604 0.018629 0.285988 0.135625 0.216996 0.248538 0.295101]
         Output = 9.54832511

Method 1: Linear Regression
------------------------------------------------------------
Method 2: Gradient Boosting
------------------------------------------------------------
Method 3: Random Forest
------------------------------------------------------------
Method 4: Neural Network
------------------------------------------------------------
Method 5: Gaussian Process
------------------------------------------------------------
Method 6: Bayesian Optimization
------------------------------------------------------------
Bayesian Optimization found: [1.    0.001 1.    0.001 0.001 1.    1.    0.001]


Predictions for candidate inputs:
Input (first 4 values)         LR          